In [6]:
import os
import json

# Folders containing your test files
folders = ["../data/low", "../data/med", "../data/high"]

# Dictionary to hold all loaded texts
texts = {}

for folder in folders:
    for filename in os.listdir(folder):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder, filename)
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)
                
                # Use filename without extension as key
                key_base = os.path.splitext(filename)[0]
                
                # Save all keys from the JSON file (like 'text', 'reference_text') inside a sub-dictionary
                texts[key_base] = {k: v for k, v in data.items()}

# Example access
print(texts["test8"]["text"][:500])
print(texts["DocumentForParsing2"]["reference_text"][:500])

A cyber-attack cut power to thousands in Killeen; investigators found malware in plant network logs and compromised vendor credentials. The mayor set up a task force to strengthen city networks.
Killeen Cyber Attack Disrupts Local Power Plant. KILLEEN, TX – On the morning of March 26th, the Killeen Power Plant experienced a cyber-attack that cut power to about 2,300 homes in Killeen. Operators detected unusual network activity at 4:00 a.m. and shut off four feeder circuits as a precaution. Workers are facilitating return of power and expect power to return at around 10:00am. What Happened Systems Hit: Main control server and backup communication lines. Impact: Widespread outages in four


In [5]:
# import nltk
# Download NLTK data once
# Vader lexicon used for sentiment analysis
# nltk.download('vader_lexicon')

In [7]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from lexicon import LEXICON as lexicon
sia = SentimentIntensityAnalyzer()

# Keep scores within [-1, 1]
def clamp(score):
    return max(-1.0, min(1.0, score))

# Avoid division by zero if no keyword hits
def safe_div(numer, denom):
    return numer / denom if denom != 0 else 0.0

def analyze_text(text):
    fear = stress = morale = trust = 0
    hits = []
    tokens = text.lower().split()
    max_fear = max_stress = max_morale = max_trust = 0

    for tok in tokens:
        if tok in lexicon:
            f, s, m, t = lexicon[tok]
            fear += f
            stress += s
            morale += m
            trust += t

            # Get max values for scaling
            max_fear += max(0, f)
            max_stress += max(0, s)
            max_morale += max(0, m)
            max_trust += max(0, t)

            hits.append((tok, (f, s, m, t)))

    comp = sia.polarity_scores(text)['compound']
    if comp > 0:
        morale += comp * 1.0
        trust += comp * 0.8
        fear -= abs(comp) * 0.3
        stress -= abs(comp) * 0.5
    elif comp < 0:
        fear += abs(comp) * 0.6
        stress += abs(comp) * 0.6
        morale -= abs(comp) * 0.5
        trust -= abs(comp) * 0.3

    return "fear: " + str(clamp(safe_div(fear, max_fear))) + ", stress: " + str(clamp(safe_div(stress, max_stress))) + ", morale: " + str(clamp(safe_div(morale, max_morale))) + ", trust: " + str(clamp(safe_div(trust, max_trust)))

In [ ]:
print("test1:", analyze_text(texts["test1"]["text"]))
print("test2:", analyze_text(texts["test2"]["text"]))
print("test3:", analyze_text(texts["test3"]["text"]))
print("test4:", analyze_text(texts["test4"]["text"]))
print("test5:", analyze_text(texts["test5"]["text"]))
print("test6:", analyze_text(texts["test6"]["text"]))
print("test7:", analyze_text(texts["test7"]["text"]))
print("test8:", analyze_text(texts["test8"]["text"]))
print("test9:", analyze_text(texts["test9"]["text"]))
print("test10:", analyze_text(texts["test10"]["text"]))
print("test11:", analyze_text(texts["test11"]["text"]))
print("test12:", analyze_text(texts["test12"]["text"]))
print("DocumentForParsing:", analyze_text(texts["DocumentForParsing"]["text"]))
print("DocumentForParsing2:", analyze_text(texts["DocumentForParsing2"]["reference_text"]))

test1: fear: 0.0, stress: 0.0, morale: 0.0, trust: 0.0
test2: fear: 1.0, stress: 1.0, morale: 0.0, trust: 0.0
test3: fear: 0.0, stress: 0.0, morale: 0.0, trust: 0.0
test4: fear: 0.0, stress: 0.0, morale: 0.0, trust: 0.0
test5: fear: 0.0, stress: 0.0, morale: 0.0, trust: 0.0
test6: fear: 1.0, stress: 1.0, morale: 0.0, trust: 0.0
test7: fear: 0.0, stress: 0.0, morale: 0.0, trust: 0.0
test8: fear: 0.0, stress: 0.0, morale: 0.0, trust: 0.0
test9: fear: 0.0, stress: 0.0, morale: 0.0, trust: 0.0
test10: fear: 1.0, stress: 1.0, morale: 0.0, trust: 0.0
test11: fear: 0.0, stress: 0.0, morale: 0.0, trust: 0.0
test12: fear: 0.0, stress: 0.0, morale: 0.5610833333333334, trust: 0.47330000000000005


KeyError: 'test13'